In [8]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [20]:
df = pd.read_csv('../data/df_model.csv')
df.head()

,age,sex,bmi,children,smoker,region,logcharges,age_x_smoker,bmi_x_smoker
0,19,1,27.900,0,1,southwest,9.734176,19,27.9
1,18,0,33.770,1,0,southeast,7.453302,0,0.0
2,28,0,33.000,3,0,southeast,8.400538,0,0.0
3,33,0,22.705,0,0,northwest,9.998092,0,0.0
4,32,0,28.880,0,0,northwest,8.260197,0,0.0


In [21]:
#train test split
X = df.drop('logcharges', axis=1)
y = df['logcharges']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)

In [22]:
#Encoding (after splitting to prevent data leakage)

ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe.fit(X_train[['region']])
X_train_region = ohe.transform(X_train[['region']])
X_test_region = ohe.transform(X_test[['region']])
region_cols = ohe.get_feature_names_out(['region'])


X_train_num = X_train.drop(columns=['region'])
X_test_num = X_test.drop(columns=['region'])

X_train_encoded = pd.concat(
    [
        X_train_num.reset_index(drop=True),
        pd.DataFrame(X_train_region, columns=region_cols).reset_index(drop=True)
    ],
    axis=1
)

X_test_encoded = pd.concat(
    [
        X_test_num.reset_index(drop=True),
        pd.DataFrame(X_test_region, columns=region_cols).reset_index(drop=True)
    ],
    axis=1
)


In [23]:
X_train_encoded.head()
X_train_encoded.shape

(1070, 10)

In [ ]:
#Linear Regression

model = LinearRegression()
model.fit(X_train_encoded, y_train)
y_pred = model.predict(X_test_encoded)




In [28]:
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'r2 score: {r2}\nMSE: {mse}')

r2 score: 0.853688392134992
MSE: 0.13155471619617218
